**1. ETL (ORIGINAL DATASET)**

# **1.1 Import the required libraries**

The following libraries are common in Python programming environments, especially in the context of data analysis and data science.

Pandas (import pandas as pd):

Pandas is a Python library that provides flexible date structures and date analysis tools. Importing it as pd is a common convention to shorten the library name and make the code more consice. Pandas is widely used to manipulate and analyze tabular datasets.

JSON(import json):

JSON (JavaScript Object Notation) is a lightweight, human-readable data interchange format. The json library in Python allows for the serialization and deserialization of data in JSON format. It can be used to read and write data in this format.

AST (import ast):

AST (Abstract Syntax Tree) is a hierarchical representation of the syntactic structure of a Python program. The ast library allows you to analyze and manipulate the abstract syntax tree of a Python source code. It can be useful for performing static analysis of the code.

Regular Expressions (import re)

The 're' module provides regular expression operations in Python. Regular expressions are search patterns used to match text strings. They are powerful tools for manipulating and searching specific patterns within text string.

CSV (import csv)

The 'csv' module in Python provides funtionality for working with CSV (Comma-Separated Values) files. CSV files are a common format for storing tabular data, where each row in the file represents a data entry and values are separated by commas or another delimiter.

The 'csv' module in Python provides functions for reading data from CSV files and writing data to CSV files. Some important functions include 'csv.reader()' for reading a CSV file and 'csv.writer()' for writing to a CSV file.

%load_ext autoreload & %autoreload 2:

These commands are specific to jupyter notebooks and are used to automatically reload modules before executing a cell '%load_ext autoreload' enables automatic module reloading, and '%autoreload 2' sets the autoreload to be more aggressive, even reloading module functions.

Warnings (import warnings):

The warnings module provides tools for controlling the warnings emitted by Python. In this case, it's being configured to ignore warnings, which can be useful to prevent warnings from filling up the console output and distracting during code execution. In summary, these imports are common in data analysis and date science environments in Python, providing tools for manipulating data, working with JSON, parsing regular expressions, and handling warnings. Additionaly, the '%load_ext autoreload and %autoreload 2' commands are specific to jupyter notebooks and are used to facilitate interactive development.

In [47]:
'''Necessary libraries.'''
import pandas as pd                 # Pandas for tabular data manipulation.
import json                         # Module for working with JSON.
import ast                          # Module for evaluating Python literal expressions.
import re                           # Module for working with regular expressions.
import csv                          # I import the CSV module into Python.

'''Enable auto-reload of modules before executing a cell'''
%load_ext autoreload
%autoreload 2

'''Import the warning module and set it to ignore all warnings'''
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# **1.2 Auxiliary Functions.**

**1.2.1 Check Data Type**

We're checking the data types contained in the columns of "df_games".

This function called 'check data types' takes a DataFrame as input and returns a new DataFrame that provides information about the data types and the quantity of null values in each column of the input DtaFrame.

This function offers a useful summary of the features of the columns in a DataFrame, incluiding data types and the quantity of null values.
It's especially useful for conducting an initial data quality analysis on a dataset.

In [48]:
def check_data_type(df):
    
    '''A dictionary (my_dict) is created with five keys: "field_name", "data_type", "non_null_%", "null_%" and "nulls". 
    These keys will be used to store information about each column of the DataFrame.'''
    
    my_dict = {"field_name" : [], "data_type" : [], "non_null_%" : [], "null_%" : [], "nulls" : []}
    
    
    '''A loop is performed over all the columns of the DataFrame df'''
    for column in df.columns:
        percentage_non_nulls = (df[column].count() / len(df) * 100)     # The percentage of non-null values in the current column is calculated 
        my_dict['field_name'].append(column)                            # The current column is added to the list under the key 'field_name'        
        my_dict['data_type'].append(df[column].apply(type).unique())    # Obtain the unique data type in the current column and add it to the list under the key 'data type'.
        my_dict['non_null_%'].append(round(percentage_non_nulls, 2))    # Add the percentage of non-null values to the list under the key 'non_null_%'.
        my_dict['null_%'].append(round(100 - percentage_non_nulls, 2))  # Add the percentage of null values to the list under the key 'null_%'.
        my_dict['nulls'].append(df[column].isnull().sum())              # Add the number of null values in the current column to the list under the key 'nulls'.
        
    '''The dictionari my_dict is used to create a new DataFrame called df_info.'''
    df_info = pd.DataFrame(my_dict)
        
    '''The function returns the DataFrame df_info containing information about each column,
    incluiding the column name, data type, percentage of non-null values, percentage of null values, and the number of null values.'''
    return df_info

**1.2.2 Check duplicates by columns**

The following function provides a useful tool for identifying and sorting duplicate rows in a pandas DataFrame based on the values of a specifict column.
In our case, it can be useful for data analysis when examining and handling duplicates based on a particular column.

In [49]:
def check_duplicates_by_columns(df, column):
    
    '''Duplicate rows are filtered'''
    duplicated_rows = df[df.duplicated(subset=column, keep=False)]
    if duplicated_rows.empty:
        return 'There are no duplicates'
    
    '''The duplicate rows are sorted for comparison'''
    duplicated_rows_sorted = duplicated_rows.sort_values(by=column)
    return duplicated_rows_sorted

**1.2.3 Replace_with_float.**  

This function attempts to convert a value to a floating-point number and returns 0.0 if the value is Nan or there is any problem during the conversion.  



In [50]:
def replace_with_float(value):
    '''Check if the value is a NaN (Not a Number) using the pandas (pd) isna function'''
    if pd.isna(value):
        '''If the value is NaN, return 0.0'''
        return 0.0
    try:
        '''Try to convert the value to a floating-point number'''
        float_value = float(value)
        '''If the conversion is successful, return the converted value'''
        return float_value
    except:
        '''If there is an exception during the conversion (e.g., if the value is not convertible to float), return 0.0'''
        return 0.0


# **1.3 ETL output_steam_games**

The code below loads data from a JSON file named output_steam_games.json into a variable named df_games.  

Reading a JSON file line by line means processing the contents of a JSON file by taking each line of the file separately. Each line of the JSON file is interpreted and converted into an object or a data structure thet can be manipulated in the programming language you are using.  

Most of the time, a JSON file is stored in a format that includes multiple JSON objects, each on a separate line. This is known as JSON per line or JSON lines (JSONL). Each line of the file is a complete and standanlone JSON object.  

In [51]:
'''The JSON file path is set to the rows_games variable.
This path points to the output_steam_games.json file in the local file system.'''
path_games = 'C:\\Users\\migue\\Proyecto_Individual_1_MLOps\\PI MLOps - STEAM\\steam_games.json\\output_steam_games.json'


'''Open the JSON file and read it line by line.  
Each line is converted from JSON to a Python dictionary using json.loads(), and the resulting dictionary is added to the rows_games list.'''

rows_games = []
with open(path_games) as f:
    for line in f.readlines():
        data = json.loads(line)
        rows_games.append(data)
        
        
'''The list of dictionaries rows_games is used to create a pandas DataFrame named df_games.
Each dictionary in the list represents a row in the DataFrame.'''
df_games = pd.DataFrame(rows_games)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


**Exploring and enhacinfg our DataFrame**  



In [52]:
'''Empty rows are removed from df_games, reducing the count from 120.445, rows to 32.135, eliminating 88.310 rows with empty values.'''
df_games = df_games.dropna(how='all').reset_index(drop=True)
df_games.shape

(32135, 13)

In [53]:
'''We verify the data types.'''
check_data_type(df_games)

,field_name,data_type,non_null_%,null_%,nulls
0,publisher,"[<class 'str'>, <class 'float'>]",74.94,25.06,8052
1,genres,"[<class 'list'>, <class 'float'>]",89.78,10.22,3283
2,app_name,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
3,title,"[<class 'str'>, <class 'float'>]",93.62,6.38,2050
4,url,[<class 'str'>],100.00,0.00,0
5,release_date,"[<class 'str'>, <class 'float'>]",93.57,6.43,2067
6,tags,"[<class 'list'>, <class 'float'>]",99.49,0.51,163
7,reviews_url,"[<class 'str'>, <class 'float'>]",99.99,0.01,2
8,specs,"[<class 'list'>, <class 'float'>]",97.92,2.08,670
9,price,"[<class 'float'>, <class 'str'>]",95.71,4.29,1377


In [54]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [55]:
'''check for duplicate values.'''
check_duplicates_by_columns(df_games, 'id')

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [56]:
'''Duplicate index are removed.'''
index_revomed = [14573, 74, 30961]
df_games = df_games.drop(index_revomed)


In [57]:
'''check for duplicate values.'''
check_duplicates_by_columns(df_games, 'id')

'There are no duplicates'

**We convert the 'release_date' column.**  

This code provides a count of the frecuency of each unique relaese date present in the 'release_date' column of the df_game DataFrame.  
It is useful for analyzing the distribution of release dates in the dataset and understanding how many times each date appears.

In [58]:
df_games['release_date'].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

Create a new column 'release_year'in the DataFrame df_games by extracting the year from the 'release_date'column.  
Then, remove the original 'release_date' column and display the first rows of the DataFrame.

In [59]:
def extract_release_year(date):
    if pd.notna(date):
        if re.match(r'^\d{4}-\d{2}-\d{2}$', date):
            return date.split('-')[0]
    return 'Unavailable data'

In [60]:
df_games['release_year'] = df_games['release_date'].apply(extract_release_year)     # Create the column with the year.

df_games = df_games.drop('release_date', axis=1)                                    # Remove column release_date

df_games.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,release_year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,Unavailable data


In [61]:
'''Let's see how unique values for "release_year" look.'''
df_games['release_year'].unique()

array(['2018', '2017', 'Unavailable data', '1997', '1998', '2016', '2006',
       '2005', '2003', '2007', '2002', '2000', '1995', '1996', '1994',
       '2001', '1993', '2004', '1999', '2008', '2009', '1992', '1989',
       '2010', '2011', '2013', '2012', '2014', '1983', '1984', '2015',
       '1990', '1988', '1991', '1985', '1982', '1987', '1981', '1986',
       '2021', '2019', '1975', '1970', '1980'], dtype=object)

In [62]:
'''We apply the transformation to the 'price' column.'''
df_games['price'] = df_games['price'].apply(replace_with_float)
df_games['price'].dtype

dtype('float64')

Now we transform the columns "publisher", "app name", "title" and "developer".  

This code block fills the nulls vaues in the specified columns with the string 'No data available' and replaces the original columns with the filled versions in the DataFrame df_games.

In [63]:
column_to_transform = ['publisher', 'app_name', 'title', 'developer']                   # Columns to transform
df_fill = df_games[column_to_transform].fillna('No data available')                     # Nulls are filled
df_games = pd.concat([df_games.drop(column_to_transform, axis=1), df_fill], axis=1)      # The proginal columns are deleted and the filled ones are concatenated with the entire DataFrame.
df_games.head()


,genres,url,tags,reviews_url,specs,price,early_access,id,release_year,publisher,app_name,title,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,2018,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,2017,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,2017,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
4,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Unavailable data,No data available,Log Challenge,No data available,No data available


This code descomposes the lists or sets nested in the 'genres' column into individual rows using explode, removes rows containing null values in 'genres', and displays the first rows of the resulting DataFrame. 

In [64]:
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=('genres'))
df_games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_year,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


When considered not to be used, columns "tags", "specs", "url" y "reviews_url" are eliminated.

In [65]:
df_games = df_games.drop(["tags", "specs", "url", "reviews_url"], axis=1)
df_games.columns

Index(['genres', 'price', 'early_access', 'id', 'release_year', 'publisher',
       'app_name', 'title', 'developer'],
      dtype='object')

In [66]:
'''We verify the data types.'''
check_data_type(df_games)

,field_name,data_type,non_null_%,null_%,nulls
0,genres,[<class 'str'>],100.0,0.0,0
1,price,[<class 'float'>],100.0,0.0,0
2,early_access,[<class 'bool'>],100.0,0.0,0
3,id,[<class 'str'>],100.0,0.0,0
4,release_year,[<class 'str'>],100.0,0.0,0
5,publisher,[<class 'str'>],100.0,0.0,0
6,app_name,[<class 'str'>],100.0,0.0,0
7,title,[<class 'str'>],100.0,0.0,0
8,developer,[<class 'str'>],100.0,0.0,0


We save the clean file

In [68]:
'''Specify directory path and CSV file name'''
directory = 'C:\\Users\\migue\\Optimizing Recommender Systems with an Advanced MLOps Pipeline\\clean_dataset'
file_name = 'Output_steam_games_clean.csv'
full = f'{directory}/{file_name}'

'''Save the DataFrame as a CSV file in the specified folder'''
df_games.to_csv(full, index=False)

print(f'The CSV file was saved in: {full}')

The CSV file was saved in: C:\Users\migue\Optimizing Recommender Systems with an Advanced MLOps Pipeline\clean_dataset/Output_steam_games_clean.csv
